In [1]:
import geopandas as gpd
import psycopg2
from shapely.wkt import loads as wkt_loads
from keys import pg_user, pg_pass, pg_host, pg_port, pg_db

output_path = 'data/listings_joined_tracts.csv'

In [2]:
connection = psycopg2.connect(database=pg_db,
                              user=pg_user,
                              password=pg_pass,
                              host=pg_host,
                              port=pg_port)
cursor = connection.cursor()

In [3]:
# verify that the SRIDs match
cursor.execute("""SELECT
                      Find_SRID('public', 'tracts', 'geom') as tracts_srid,
                      Find_SRID('public', 'listings', 'geom') as listings_srid""")
cursor.fetchall()

[(4326, 4326)]

In [4]:
# joining how many rows to how many rows?
cursor.execute("SELECT count(*) AS row_count FROM tracts")
count_tracts_table = cursor.fetchall()[0][0]
cursor.execute("SELECT count(*) AS row_count FROM listings")
count_listings_table = cursor.fetchall()[0][0]
count_tracts_table, count_listings_table

(74133, 1393426)

## Do the spatial join

In [5]:
%%time
# drop the table if it already exists and recreate it
# select distinct to keep only one listing/tract pair even if the listing lies on the border of multiple tracts
query = """
        DROP TABLE IF EXISTS listings_tracts;
        CREATE TABLE listings_tracts AS
        SELECT DISTINCT ON (listing_id)
            listings.id AS listing_id,
            listings.date AS date,
            listings.region AS region,
            listings.bedrooms AS bedrooms,
            listings.rent AS rent,
            listings.sqft AS sqft,
            listings.geom AS geometry,
            tracts.geoid AS tract_geoid
        FROM listings LEFT JOIN tracts
        ON ST_Intersects(listings.geom, tracts.geom)
        """

cursor.execute(query)
connection.commit()

Wall time: 1min 33s


In [6]:
%%time
# vacuum and analyze the database to optimize it after building indices and inserting rows
old_isolation_level = connection.isolation_level
connection.set_isolation_level(0)
cursor.execute("VACUUM ANALYZE")
connection.commit()
connection.set_isolation_level(old_isolation_level)

Wall time: 1.88 s


## Retrieve the data

In [7]:
# some listings still lack tract geoid, as they are offshore or just across canada/mexico border
cursor.execute("SELECT count(*) AS row_count, count(tract_geoid) AS geoid_count FROM listings_tracts")
print(count_listings_table)
cursor.fetchall()

1393426


[(1393426, 1392866)]

In [8]:
%%time
# select the results from the new table, ignoring any listings with null tract_geoid
# because they are offshore or just across canada/mexico border
cursor.execute("""SELECT date, region, bedrooms, rent, sqft, ST_AsText(geometry), tract_geoid
                  FROM listings_tracts
                  WHERE tract_geoid IS NOT NULL""")
rows = cursor.fetchall()

Wall time: 5.84 s


In [9]:
cursor.close()
connection.close()

In [10]:
%%time
# turn the selected rows into a geodataframe
gdf = gpd.GeoDataFrame(rows, columns=['date', 'region', 'bedrooms', 'rent', 'sqft', 'geometry', 'tract_geoid'])
gdf['geometry'] = gdf['geometry'].map(lambda x: wkt_loads(x))
gdf['lat'] = gdf['geometry'].map(lambda pt: pt.y)
gdf['lng'] = gdf['geometry'].map(lambda pt: pt.x)
gdf = gdf.drop('geometry', axis=1)

Wall time: 53.7 s


In [11]:
print(len(gdf))
gdf.head()

1392866


,date,region,bedrooms,rent,sqft,tract_geoid,lat,lng
0,2014-05-11,santabarbara,3.0,3500.0,1200.0,06083001304,34.399757,-119.726987
1,2014-05-11,santabarbara,2.0,850.0,882.0,06083002924,34.411019,-119.855845
2,2014-05-11,santabarbara,1.0,1290.0,NaN,06083002924,34.410415,-119.854330
3,2014-05-11,santabarbara,1.0,1660.0,NaN,06083000102,34.441460,-119.754324
4,2014-05-11,santabarbara,2.0,3200.0,NaN,06083002924,34.410416,-119.854530


In [12]:
%%time
gdf.to_csv(output_path, encoding='utf-8', index=False)

Wall time: 10 s
